# 02_CHICAGO_BULLS_test.ipynb

In [1]:
## import と基本設定
import os
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster, leaguedashplayerstats
from requests.exceptions import RequestException

# 2024-25シーズン表記（NBA公式APIのSeasonパラメータ形式）
SEASON = "2024-25"

# シカゴ・ブルズの略称
TEAM_ABBR = "CHI"

# 必要なら出力先フォルダ（任意）
OUTPUT_DIR = "data/bulls"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Setup done.")


Setup done.


In [ ]:
## チームID（team_id）を取得
## ここで team_id が取れればOK
def get_team_id_by_abbr(team_abbr: str) -> int:
    """チーム略称（例: CHI）から team_id を取得."""
    all_teams = teams.get_teams()
    for t in all_teams:
        if t.get("abbreviation") == team_abbr:
            return t["id"]
    raise ValueError(f"Team abbreviation not found: {team_abbr}")

try:
    team_id = get_team_id_by_abbr(TEAM_ABBR)
    print(f"Team: {TEAM_ABBR}, team_id: {team_id}")
except Exception as e:
    print("チームID取得中にエラー:", e)
    raise


Team: CHI, team_id: 1610612741


In [3]:
## ロスター情報を取得（CommonTeamRoster）
def fetch_team_roster(team_id: int, season: str) -> pd.DataFrame:
    """
    指定チーム・シーズンのロスター情報を取得。
    エラーハンドリング付き。
    """
    try:
        res = commonteamroster.CommonTeamRoster(
            team_id=team_id,
            season=season,
            timeout=60,   # 念のため長め
        )
        df_roster = res.get_data_frames()[0]
        print(f"Roster rows: {len(df_roster)}")
        return df_roster
    except RequestException as e:
        print("HTTPレベルのエラーが発生しました:", e)
        raise
    except Exception as e:
        print("ロスター取得中に予期せぬエラーが発生しました:", e)
        raise

df_roster = fetch_team_roster(team_id, SEASON)
df_roster.head()


Roster rows: 18


,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612741,2024,00,Coby White,Coby,coby-white,0,G,6-5,195,"FEB 16, 2000",25.0,5,North Carolina,1629632,#7 Pick in 2019 Draft
1,1610612741,2024,00,Lonzo Ball,Lonzo,lonzo-ball,2,G,6-6,190,"OCT 27, 1997",27.0,7,UCLA,1628366,None
2,1610612741,2024,00,Josh Giddey,Josh,josh-giddey,3,G,6-8,216,"OCT 10, 2002",22.0,3,NBA Global Academy,1630581,Traded from OKC on 06/21/24
3,1610612741,2024,00,Jevon Carter,Jevon,jevon-carter,5,G,6-1,200,"SEP 14, 1995",29.0,6,West Virginia,1628975,Signed on 07/10/23
4,1610612741,2024,00,Jalen Smith,Jalen,jalen-smith,7,F-C,6-9,215,"MAR 16, 2000",25.0,4,Maryland,1630188,Signed on 07/08/24


In [4]:
## シーズンスタッツを取得（LeagueDashPlayerStats）
def fetch_team_player_stats(team_id: int, season: str) -> pd.DataFrame:
    """
    チームの選手シーズンスタッツを取得。
    leaguedashplayerstats で team_id_nullable を指定。
    """
    try:
        stats = leaguedashplayerstats.LeagueDashPlayerStats(
            season=season,
            team_id_nullable=team_id,
            timeout=60,
        )
        df_stats = stats.get_data_frames()[0]
        print(f"Stats rows: {len(df_stats)}")
        return df_stats
    except RequestException as e:
        print("HTTPレベルのエラーが発生しました:", e)
        raise
    except Exception as e:
        print("スタッツ取得中に予期せぬエラーが発生しました:", e)
        raise

df_stats = fetch_team_player_stats(team_id, SEASON)
df_stats.head()


Stats rows: 22


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT
0,1641766,Adama Sanogo,Adama,1610612741,CHI,23.0,4,2,2,0.500,...,3,1,19,22,9,22,11,4,22,1
1,1630245,Ayo Dosunmu,Ayo,1610612741,CHI,25.0,46,17,29,0.370,...,18,16,9,6,22,8,7,2,8,1
2,1630537,Chris Duarte,Chris,1610612741,CHI,28.0,17,4,13,0.235,...,3,4,17,18,7,18,11,4,18,1
3,1629632,Coby White,Coby,1610612741,CHI,25.0,74,35,39,0.473,...,22,21,1,1,19,3,6,4,2,1
4,1631207,Dalen Terry,Dalen,1610612741,CHI,22.0,73,33,40,0.452,...,13,18,11,12,20,10,11,4,10,1


In [5]:
## ロスター＋スタッツを結合して「情報一式」にする
# 結合用に key を確認
print("Roster columns:", df_roster.columns.tolist())
print("Stats columns:", df_stats.columns.tolist())

# 多くのケースで 'PLAYER_ID' が共通キー
merge_key = "PLAYER_ID"

df_all = pd.merge(
    df_roster,
    df_stats,
    on=merge_key,
    how="left",
    suffixes=("_ROSTER", "_STATS"),
)

print(df_all.shape)
df_all.head()


Roster columns: ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Stats columns: ['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'TEAM_COUNT']
(18, 82)


,TeamID,SEASON,LeagueID,PLAYER,NICKNAME_ROSTER,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT
0,1610612741,2024,00,Coby White,Coby,coby-white,0,G,6-5,195,...,22,21,1,1,19,3,6,4,2,1
1,1610612741,2024,00,Lonzo Ball,Lonzo,lonzo-ball,2,G,6-6,190,...,7,10,15,13,2,11,11,4,11,1
2,1610612741,2024,00,Josh Giddey,Josh,josh-giddey,3,G,6-8,216,...,21,19,2,3,17,2,2,1,3,1
3,1610612741,2024,00,Jevon Carter,Jevon,jevon-carter,5,G,6-1,200,...,7,6,16,16,4,16,11,4,16,1
4,1610612741,2024,00,Jalen Smith,Jalen,jalen-smith,7,F-C,6-9,215,...,11,15,5,8,15,6,3,4,7,1
